In [2]:
# =====================================================
# BASELINE: LOGISTIC REGRESSION - WELFAKE
# Output Format: HuggingFace Style
# =====================================================

import os, re, psutil, pickle, time
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from google.colab import drive

# 1. MOUNT DRIVE & SETUP
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive', force_remount=True)
    except ValueError: pass

OUTPUT_DIR = "/content/drive/MyDrive/WELFake_LogReg_Baseline"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. LOAD & CLEAN DATA
print("⏳ Đang tải dataset WELFake...")
dataset = load_dataset("davanstrien/WELFake")
df = pd.DataFrame(dataset["train"])

def clean_text_ml(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', '', s)
    s = re.sub(r'<.*?>', '', s)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Pre-processing...")
df['content'] = (df['title'].fillna('') + " " + df['text'].fillna('')).apply(clean_text_ml)
df = df[df['content'].str.len() > 50]

# 3. SPLIT
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['content'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

# 4. TF-IDF
print("⚙️ Vectorizing (TF-IDF)...")
vectorizer = TfidfVectorizer(max_features=50000, stop_words='english', ngram_range=(1, 2))
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

# 5. TRAIN
print("🚀 Training Logistic Regression...")
model = LogisticRegression(solver='liblinear', C=1.0, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# =====================================================
# 6. EVALUATION (HuggingFace Style Format)
# =====================================================
print("\n🎯 ĐANG ĐÁNH GIÁ (TEST SET)...")

# Bắt đầu đo thời gian dự đoán
start_time = time.time()

# Dự đoán nhãn và xác suất
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1] # Lấy xác suất lớp 1 (Real)

# Kết thúc đo thời gian
end_time = time.time()
runtime = end_time - start_time
samples_per_second = len(y_test) / runtime

# Tính toán các chỉ số
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
auc = roc_auc_score(y_test, y_prob)

# Tạo dictionary kết quả giống HuggingFace
eval_results = {
    'eval_accuracy': accuracy,
    'eval_precision': precision,
    'eval_recall': recall,
    'eval_f1': f1,
    'eval_auc': auc,
    'eval_loss': 'N/A (LogReg)', # Logistic Regression sklearn không trả về loss theo epoch như DL
    'eval_runtime': runtime,
    'eval_samples_per_second': samples_per_second,
    'eval_steps_per_second': 'N/A' # Không áp dụng cho sklearn
}

print("\n" + "="*50)
print("📊 KẾT QUẢ CUỐI CÙNG:")
print("="*50)
print(eval_results)
print("="*50)

# 7. SAVE
with open(os.path.join(OUTPUT_DIR, "logreg_model.pkl"), "wb") as f:
    pickle.dump(model, f)
with open(os.path.join(OUTPUT_DIR, "tfidf_vectorizer.pkl"), "wb") as f:
    pickle.dump(vectorizer, f)
print(f"\n✅ Đã lưu model tại: {OUTPUT_DIR}")

⏳ Đang tải dataset WELFake...
🧹 Pre-processing...
⚙️ Vectorizing (TF-IDF)...
🚀 Training Logistic Regression...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  warnings.warn(



🎯 ĐANG ĐÁNH GIÁ (TEST SET)...

📊 KẾT QUẢ CUỐI CÙNG:
{'eval_accuracy': 0.961310973488275, 'eval_precision': 0.9613805053894593, 'eval_recall': 0.961310973488275, 'eval_f1': 0.9613027825808775, 'eval_auc': np.float64(0.9934692191873555), 'eval_loss': 'N/A (LogReg)', 'eval_runtime': 0.01752948760986328, 'eval_samples_per_second': 819818.6005113976, 'eval_steps_per_second': 'N/A'}

✅ Đã lưu model tại: /content/drive/MyDrive/WELFake_LogReg_Baseline
